


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [129]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [117]:
from google.colab import files
uploaded = files.upload()

Saving corpus.txt to corpus (2).txt


In [237]:
# Load the data and preprocess data and store corpus in raw_text
import os
import re
filename = "corpus.txt"
fd = open(filename ,encoding="utf8")
text = fd.read()
data = text 
tmp = ''

text = text.lower()                               #convert to each alphabet lower case
text = text.split('\n')                           #split to sentences          
for i in range(len(text)):                        #remove non alphanumeric by using regex
    text[i] = re.sub(r'[^\w\s]', ' ', text[i])
    text[i] = re.sub(r'\w*\d\w*', ' ', text[i])
    tmp += ' ' + text[i] + '\n'
text = tmp                                        #get combined sentence 
print(text)

 chapter i 
 down the rabbit hole alice was beginning to get very tired of sitting by her sister on the bank  and of having nothing to do  once or twice she had peeped into the book her sister was reading  but it had no pictures or conversations in it   and what is the use of a book   thought alice  without pictures or conversations   so she was considering in her own mind  as well as she could  for the hot day made her feel very sleepy and stupid   whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies  when suddenly a white rabbit with pink eyes ran close by her 
 there was nothing so  very  remarkable in that  nor did alice think it so  very  much out of the way to hear the rabbit say to itself   oh dear 
 oh dear 
 i shall be late    when she thought it over afterwards  it occurred to her that she ought to have wondered at this  but at the time it all seemed quite natural   but when the rabbit actually  took a watch out of its 

In [203]:
# Hyperparameters of the model
vocab_size = 2500 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [204]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index

In [205]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([text])[0]
print(tokens)
print(len(tokens))
print(len(set(tokens)))

[308, 8, 39, 2, 93, 598, 12, 14, 257, 4, 107, 31, 476, 9, 349, 78, 17, 385, 21, 2, 832, 3, 9, 350, 135, 4, 55, 136, 58, 599, 7, 24, 833, 66, 2, 326, 17, 385, 14, 834, 25, 6, 24, 46, 689, 58, 1061, 13, 6, 3, 32, 37, 2, 212, 9, 5, 326, 61, 12, 171, 689, 58, 1061, 27, 7, 14, 835, 13, 17, 351, 327, 16, 68, 16, 7, 59, 26, 2, 477, 156, 151, 17, 422, 31, 600, 3, 536, 328, 2, 1062, 9, 423, 5, 1459, 1460, 51, 29, 690, 2, 537, 9, 189, 40, 3, 1063, 2, 1461, 57, 283, 5, 152, 93, 23, 1462, 157, 231, 284, 78, 17, 41, 14, 135, 27, 31, 1064, 13, 15, 836, 69, 12, 90, 6, 27, 31, 94, 36, 9, 2, 84, 4, 258, 2, 93, 95, 4, 259, 109, 158, 109, 158, 8, 176, 29, 538, 57, 7, 61, 6, 120, 1065, 6, 1066, 4, 17, 15, 7, 260, 4, 56, 1463, 19, 33, 25, 19, 2, 63, 6, 22, 165, 87, 691, 25, 57, 2, 93, 1464, 182, 5, 424, 36, 9, 81, 1067, 478, 3, 110, 19, 6, 3, 44, 329, 21, 12, 1068, 4, 17, 205, 26, 6, 1465, 601, 17, 327, 15, 7, 24, 105, 130, 241, 5, 93, 23, 352, 5, 1067, 478, 58, 5, 424, 4, 190, 36, 9, 6, 3, 1466, 23, 602, 

In [206]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  27213


In [207]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [209]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 50, 100)           250000    
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               84480     
_________________________________________________________________
dense_11 (Dense)             (None, 2500)              322500    
Total params: 656,980
Trainable params: 656,980
Non-trainable params: 0
_________________________________________________________________


In [210]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=120, batch_size=128, validation_split = 0.2 ) 

Epoch 1/120
171/171 [==============================] - 4s 21ms/step - loss: 6.3424 - accuracy: 0.0514 - val_loss: 6.1317 - val_accuracy: 0.0829
Epoch 2/120
171/171 [==============================] - 3s 17ms/step - loss: 5.8784 - accuracy: 0.0543 - val_loss: 6.0626 - val_accuracy: 0.0829
Epoch 3/120
171/171 [==============================] - 3s 17ms/step - loss: 5.7632 - accuracy: 0.0586 - val_loss: 6.0492 - val_accuracy: 0.0841
Epoch 4/120
171/171 [==============================] - 3s 17ms/step - loss: 5.6689 - accuracy: 0.0615 - val_loss: 6.0518 - val_accuracy: 0.0889
Epoch 5/120
171/171 [==============================] - 3s 17ms/step - loss: 5.5790 - accuracy: 0.0673 - val_loss: 6.0113 - val_accuracy: 0.0863
Epoch 6/120
171/171 [==============================] - 3s 17ms/step - loss: 5.4816 - accuracy: 0.0737 - val_loss: 5.9939 - val_accuracy: 0.0950
Epoch 7/120
171/171 [==============================] - 3s 17ms/step - loss: 5.3696 - accuracy: 0.0849 - val_loss: 5.9472 - val_accuracy:

In [212]:
#reverse_word_map = ##  word to idx map using tokenizeCreater.word_index
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [213]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n):
	final_string = ''
	print ("Seed: \n",  input_str)
	X1 =  numpy.array(tokenizer.texts_to_sequences([input_str])[0])
	for i in range(n):
		X1 = numpy.reshape(X1, (1, len(X1), 1))					#convert to input shape batchsizeXinput size
		prediction =  numpy.argmax(model.predict( X1 , verbose=0))
		next_word = reverse_word_map[prediction]
		final_string += ' '+next_word
		X1 = numpy.append(X1,prediction)
		X1 = X1[1:]
	return final_string

In [214]:
import random
# pick a random seed
random.seed(30)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])
print("next generated words:\n",next_tokens( input_str , 10))

Seed: 
 of canterbury found it advisable found what said the duck found it the mouse replied rather crossly of course you know what it means i know what it means well enough when i find a thing said the duck it s generally a frog or a worm the question is
next generated words:
  what did the archbishop find the mouse did not notice


In [218]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print("next generated words:\n",next_tokens( text_to_word_sequence(input_str)[0:50] , 10))

Seed: 
 ['the', 'boy', 'laughed', 'at', 'the', 'fright', 'he', 'had', 'caused', 'this', 'time', 'the', 'villagers', 'left', 'angrily', 'the', 'third', 'day', 'as', 'the', 'boy', 'went', 'up', 'the', 'small', 'hill', 'he', 'suddenly', 'saw', 'a', 'wolf', 'attacking', 'his', 'sheep', 'he', 'cried', 'as', 'hard', 'as', 'he', 'could', '“wolf', 'wolf', 'wolf', '”', 'but', 'not', 'a', 'single', 'villager']
next generated words:
  in the same words a pair of the queen and


# Character based LSTM Model 1

In [281]:
# User the preprocess data and create raw_text
text = data
text = text.replace('_', ' ')
text = text.replace('-', ' ')         #only remove _ and -. Kept remaining punctutions to use them 
text = text.lower()                 
# create mapping of unique characters to integers
ch = sorted(list(set(text)))

char_to_int = dict((x, n) for n, x in enumerate(ch))

In [243]:
# Print the total characters and character vacob size
n_chars = len(text)
n_vocab = len(chars)
print(n_chars)
print(n_vocab)

142477
44


In [244]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = text[i:i + seq_length]
  seq_out = text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  142377


In [245]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX)
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [246]:
embedding_dim =100
max_length =100

In [247]:
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 100, 100)          4400      
_________________________________________________________________
lstm_13 (LSTM)               (None, 256)               365568    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 44)                11308     
Total params: 381,276
Trainable params: 381,276
Non-trainable params: 0
_________________________________________________________________


In [248]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1113/1113 [==============================] - 20s 18ms/step - loss: 2.2115
Epoch 2/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.7125
Epoch 3/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.5153
Epoch 4/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.3924
Epoch 5/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.3137
Epoch 6/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.2521
Epoch 7/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.2038
Epoch 8/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1563
Epoch 9/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1176
Epoch 10/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0826
Epoch 11/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0528
Epoch 12/20
1113/1113 [==============================] - 20s 18

In [282]:
#implement mapping of integer to character
int_to_char = dict((j, ch) for j, ch in enumerate(chars))

In [250]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	result = ''
	X1 = []
	X1.append([char_to_int[char] for char in pattern])
	for i in range(x):
		# Complete Code
		X1 = numpy.reshape(X1, (1, len(pattern), 1))
		prediction = model.predict(X1, verbose=0)
		index = numpy.argmax(prediction)
		result += int_to_char[index]
		X1 = numpy.append(X1,index)
		X1 = X1[1:len(X1)]
	return result

In [254]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print('input string: ',input_str)
print('next generated: ',predict_next_100_chars(input_str,200))

input string:  have been changed for mabel!
i’ll try and say ‘ how doth the little —’” and she crossed her hands on
next generated:   the table, and she had not a bit hurt a brown of the sea, and she tried to be sure to say the things are to her feet high.
“what did they live at the table, you know,” said the king, “i wonder what i


In [266]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."
input_str = input_str.lower()
print('input string: ',input_str[0:100])
print('next generated: ',predict_next_100_chars(input_str[0:100],200))

input string:  the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
next generated:  nd the trial doesn’t got to see it head mouth one of the little golden key, and the white rabbit was silent, and then the dormouse shook its heads down, and the whole party shark any of them shrinking


## Character based LSTM Model 2


In [280]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
model1.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 100, 100)          4400      
_________________________________________________________________
lstm_22 (LSTM)               (None, 100, 256)          365568    
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 44)                11308     
Total params: 906,588
Trainable params: 906,588
Non-trainable params: 0
_______________________________________________

In [270]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2225/2225 [==============================] - 51s 23ms/step - loss: 2.0814
Epoch 2/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.5285
Epoch 3/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.3559
Epoch 4/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.2566
Epoch 5/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.1902
Epoch 6/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.1399
Epoch 7/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.0930
Epoch 8/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.0582
Epoch 9/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.0250
Epoch 10/20
2225/2225 [==============================] - 50s 23ms/step - loss: 0.9996
Epoch 11/20
2225/2225 [==============================] - 50s 23ms/step - loss: 0.9720
Epoch 12/20
2225/2225 [==============================] - 50s 23

In [271]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	result = ''
	X1 = []
	X1.append([char_to_int[char] for char in pattern])
	for i in range(x):
		# Complete Code
		X1 = numpy.reshape(X1, (1, len(pattern), 1))
		prediction = model1.predict(X1, verbose=0)
		index = numpy.argmax(prediction)
		result += int_to_char[index]
		X1 = numpy.append(X1,index)
		X1 = X1[1:len(X1)]

	return result

In [274]:
# pick a random seed
random.seed(30)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print('input string: ',input_str)
print('next generated: ',predict_next_100_chars(input_str,200))

input string:  please your majesty,” said the knave, “i didn’t write it, and they can’t prove i did: there’s no nam
next generated:  e of the sea,” said the king, “and then it was a little bit.
so they were much my life!” “i haven’t asked the sort of thing i should like the truals of the sea.” “i don’t know what they’re a little bo


In [277]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
    a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
    him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str = input_str.lower()
print('input string: ',input_str[0:100])
print('next generated: ',predict_next_100_chars(input_str[0:100],200))

input string:  the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
next generated:  nd she soon made out that she had not a moment that she had been any more than a large cat of kind of her voice, and she thought it must be the right size with the words “off with her head!
off with h


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** Based on the output, we observe that the accuray on train data is much higher than the accuracy on the unseen data.  

The test data accuray increases initially for some epochs and then decreases (overfitting issue). The train set accuray keeps increasing with epochs.   

The loss in the trainset keeps decreasing with epochs. However, the loss in the unseen data increases with number of epochs. 

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:**
LSTM1 has single LSTM layer with dropout. LSTM2 has two LSTM layers. The number of parameters and hence computation time is more is case of LSTM2  
                                    
                                 
Time per each epoch in LSTM1: 20 secs(approx)  
Time per each epoch in LSTM2: 50 secs(approx)

The loss after 20 epochs in LSTM1: 0.8510  
The loss after 20 epochs on LSTM2: 0.8456

The LSTM 2 is better trained in same number of epochs. 

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)